In [1]:
import pandas as pd
import numpy as np
import math
import os
import spotipy

from IPython.display import display
pd.options.display.max_columns = None

This is a feature engineering script that generates variables that can be inferred from the Spotify Top 200 Charts in the US.

### Import raw Spotify charts data

Import the raw dataset of all songs on the Spotify Top 200 Charts since 2017 (assembled by Brunnell), regardless if they feature a collaboration or not. To generate artist-specific variables on the number of appearances/songs on the chart, we need to have a complete history of each artist, including all the songs they have released without collaborations.

In [2]:
data_top200 = pd.read_csv('../input/spotify_top200.csv')\
.drop_duplicates()
data_top200['Date'] = pd.to_datetime(data_top200['Date'])

In [3]:
data_us = data_top200[data_top200['Country']=='United States']\
.sort_values(by=['Artist', 'Date', 'Track Name'])
data_us['Track URI'] = data_us['Track URL'].str.replace(r'https://open.spotify.com/track/', '')

Conduct a few sanity checks to make sure the date range is complete.

In [4]:
print(max(data_us['Date']))
print(min(data_us['Date']))

2020-12-07 00:00:00
2017-01-01 00:00:00


Check that 200 unique positions per day x 1404 unique days (between 2017-01-01 and 2020-12-07) = 280600 observations in the US.

In [ ]:
print(len(data_us.index))

x = data_us.groupby('Date')[['Position']].transform('count')
print(np.max(x))
print(np.min(x))

y = data_us.groupby('Position')[['Date']].transform('count')
print(np.max(y))
print(np.min(y))

### Setting up Spotipy Credentials

In [5]:
creds = pd.read_csv('../input/credentials.csv')
SPOTIPY_CLIENT_ID = creds['Client ID'][0]
SPOTIPY_CLIENT_SECRET = creds['Client Secret'][0]

client_credentials_manager = spotipy.SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, 
                                                              client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Collect track information from the API

In [6]:
track_uris = data_us['Track URI'].unique()
len(track_uris)

7313

For each unique track in data_us, download the list of artists/collaborators involved, their names, album id, and their release date from the API.

In [86]:
def get_track_data(track_uris, n):
    
    track_artist_uris = []
    track_artist_names = []
    track_album_uri = []
    track_album_release = []
    track_album_genre = []
    loops = 0
    
#     while n* (loops+1) <= len(track_uris):
    for loops in range(math.ceil(len(track_uris)/n)):
        # getting track information in batches of size n
        temp = [x for x in sp.tracks(track_uris[n*loops : n*(loops + 1)])['tracks']]
    
        # for each track in this batch, obtain album info and list of artists
        for i in range(len(temp)):
            track_album_uri.append(temp[i]['album']['id'])
            track_album_release.append(temp[i]['album']['release_date'])
#             track_album_genre.append(sp.album(temp[i]['album']['id'])['genres'])
        
            t = temp[i]['artists']
            temp_uris = []
            temp_names = []
        
            # for each artist, obtain id and name
            for j in range(len(t)):
                temp_uris.append(t[j]['id']) 
                temp_names.append(t[j]['name'])
                
            track_artist_uris.append(temp_uris)
            track_artist_names.append(temp_names)
        
        loops += 1
        print(f'Found information for {n*(loops+1)} songs')
    
    # Combine all lists into a dataframe
    track_data = pd.DataFrame(list(zip(track_uris, track_artist_uris, track_artist_names,
                                       track_album_uri, track_album_release)),
                              columns=['Track URI', 'Artist URI', 'Artist Name',
                                       'Album URI', 'Album Release'])
    
    # Count number of collaborators per song 
    track_data['No. of Artists'] = track_data['Artist URI'].astype(str).str.count(',')+1 
    return(track_data)
#     return(temp)

In [87]:
track_data = get_track_data(track_uris, n=50)
# track_data = pd.read_csv('../output/2021.01.01 Track Artist URI 7313.csv')\
# .drop(columns = ['Unnamed: 0'])

Found information for 100 songs
Found information for 150 songs
Found information for 200 songs
Found information for 250 songs
Found information for 300 songs
Found information for 350 songs
Found information for 400 songs
Found information for 450 songs
Found information for 500 songs
Found information for 550 songs
Found information for 600 songs
Found information for 650 songs
Found information for 700 songs
Found information for 750 songs
Found information for 800 songs
Found information for 850 songs
Found information for 900 songs
Found information for 950 songs
Found information for 1000 songs
Found information for 1050 songs
Found information for 1100 songs
Found information for 1150 songs
Found information for 1200 songs
Found information for 1250 songs
Found information for 1300 songs
Found information for 1350 songs
Found information for 1400 songs
Found information for 1450 songs
Found information for 1500 songs
Found information for 1550 songs
Found information for 1600 s

### Collect artist information from the API

In [58]:
artist_uris = pd.unique(track_data['Collaborator URI'].explode())
len(artist_uris)

1611

In [71]:
def get_artist_data(artist_uris, n):
    
    artist_genres = []
    loops = 0
    
    # getting artist information in batches of size n
    for loops in range(math.ceil(len(artist_uris)/n)):
        temp = [x for x in sp.artists(artist_uris[n*loops : n*(loops + 1)])['artists']]
    
        # for each artist in this batch, obtain genre
        for i in range(len(temp)):
            artist_genres.append(temp[i]['genres'])
            
        loops += 1
        print(f'Found information for {n*(loops+1)} artists')

    artist_data = pd.DataFrame(list(zip(artist_uris, artist_genres)),
                              columns=['Artist URI', 'Artist Genre'])
    return(artist_data)

In [73]:
artist_data = get_artist_data(artist_uris, n=50)

Found information for 100 artists
Found information for 150 artists
Found information for 200 artists
Found information for 250 artists
Found information for 300 artists
Found information for 350 artists
Found information for 400 artists
Found information for 450 artists
Found information for 500 artists
Found information for 550 artists
Found information for 600 artists
Found information for 650 artists
Found information for 700 artists
Found information for 750 artists
Found information for 800 artists
Found information for 850 artists
Found information for 900 artists
Found information for 950 artists
Found information for 1000 artists
Found information for 1050 artists
Found information for 1100 artists
Found information for 1150 artists
Found information for 1200 artists
Found information for 1250 artists
Found information for 1300 artists
Found information for 1350 artists
Found information for 1400 artists
Found information for 1450 artists
Found information for 1500 artists
Fou

In [75]:
artist_data

,Artist URI,Artist Genre
0,1VPmR4DJC1PlOtd0IADAO0,"[dark trap, new orleans rap, underground hip hop]"
1,6Ff53KvcvAj5U7Z1vojB5o,"[boy band, dance pop, europop, pop]"
2,6PfSUFtkMVoDkx4MQkzOi3,"[glitchcore, hyperpop]"
3,4UXqAaa6dQYAk18Lv7PEgX,"[emo, modern rock, pop punk]"
4,5aYf0AInMznHfXGaemKEBv,[]
...,...,...
1606,3QJUFtGBGL05vo0kCJZsmT,"[modern indie pop, social media pop]"
1607,3h1fFIofdTGrHbqisHyWgI,[]
1608,6IhKl7lqJc5omkhODEJinj,[]
1609,6IrDpI3xcuzTUiEc3fnc9H,[]


In [43]:
# sp.track('4HBZA5flZLE435QTztThqH')['album']

In [42]:
# sp.album('5mUdh6YWnUvf0MfklEk1oi')['genres']

In [41]:
# sp.artist('66CXWjxzNUsdJxJ2JdwvnR')['genres']

In [88]:
# track_data.to_csv('../output/2021.01.04 Track Artist URI 7313.csv')
# artist_data.to_csv('../output/2021.01.04 Artist Genres 1611.csv')

### Generate variables at a song/track level

Generate variables on the timing of the song's release. 

In [89]:
track_data['Album Release'] = pd.to_datetime(track_data['Album Release'])
track_data['Album_release_month'] = track_data['Album Release'].dt.strftime('%m')
track_data['Album_release_dayweek'] = track_data['Album Release'].dt.strftime('%a')

In [90]:
track_data

,Track URI,Artist URI,Artist Name,Album URI,Album Release,No. of Artists,Album_release_month,Album_release_dayweek
0,4nutwPQrK56fFmrAMgyPhz,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
1,7v2azTfke2BR57lh2HxPQo,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
2,0fyBYsrmpihh1mfalssDlB,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
3,6VFKlX5qzxwmIiezqeqNYG,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
4,53AiGAa0Qi2VbX7eUpur1U,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
...,...,...,...,...,...,...,...,...
7308,3RXkboS74UYzN14xTqzPyY,[6IhKl7lqJc5omkhODEJinj],[],4yQC726OERjevM2YKtORnm,2017-07-14,1,07,Fri
7309,3bVbQvGVIe4n24AzyXovXh,[3h1fFIofdTGrHbqisHyWgI],[],6zP4EpOz9M0vhrjp7FNwKN,2017-07-17,1,07,Mon
7310,4JAyIDXOqNM6qHuZML01uX,[3h1fFIofdTGrHbqisHyWgI],[],6zP4EpOz9M0vhrjp7FNwKN,2017-07-17,1,07,Mon
7311,6Br5mChPdgQNmLF0G0gjPH,[6IrDpI3xcuzTUiEc3fnc9H],[],3jd8KzVKriW0uzSsIwxfZM,2017-07-18,1,07,Tue


Check that no songs were missed from the original list of track_uris.

In [53]:
setdiff = np.setdiff1d(track_uris, track_data['Track URI'])
len(setdiff)

0

Confirm that there are 2843 songs with collaborations, as Brunnell has found previously.

In [ ]:
bru = pd.read_csv('../input/spotify_top200_reduced.csv').drop_duplicates()
bru_us = bru[bru['Country']=='United States']

uri1 = track_data.loc[track_data['No. of Collaborators']!= 1,'Track URI'].drop_duplicates()
uri2 = bru_us['Track URI'].drop_duplicates()
print(len(uri1))
print(len(uri2))
print(len(np.setdiff1d(uri1, uri2)))
print(len(np.intersect1d(uri1, uri2)))

Merge data_us onto track_data

In [91]:
data1 = data_us.merge(track_data, how = 'left', on = 'Track URI')

In [92]:
data1

,Date,Track URL,Position,Track Name,Artist,Streams,Country,Track URI,Artist URI,Artist Name,Album URI,Album Release,No. of Artists,Album_release_month,Album_release_dayweek
0,2018-09-07,https://open.spotify.com/track/4nutwPQrK56fFmr...,196,"10,000 Degrees",$uicideBoy$,232152,United States,4nutwPQrK56fFmrAMgyPhz,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
1,2018-09-07,https://open.spotify.com/track/7v2azTfke2BR57l...,177,Bring Out Your Dead,$uicideBoy$,247359,United States,7v2azTfke2BR57lh2HxPQo,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
2,2018-09-07,https://open.spotify.com/track/0fyBYsrmpihh1mf...,146,King Tulip,$uicideBoy$,270299,United States,0fyBYsrmpihh1mfalssDlB,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
3,2018-09-07,https://open.spotify.com/track/6VFKlX5qzxwmIie...,190,Meet Mr. NICEGUY,$uicideBoy$,236054,United States,6VFKlX5qzxwmIiezqeqNYG,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
4,2018-09-07,https://open.spotify.com/track/53AiGAa0Qi2VbX7...,140,Nicotine Patches,$uicideBoy$,279299,United States,53AiGAa0Qi2VbX7eUpur1U,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280595,2017-11-10,https://open.spotify.com/track/1YqcGlCHNquxBhl...,182,NaN,NaN,205945,United States,1YqcGlCHNquxBhlUZsjhMT,[2uwXb02RFgtGOr6s6XMWlB],[],2oxjmU5c2mZzc7mSQqWTE9,2017-11-07,1,11,Tue
280596,2017-11-11,https://open.spotify.com/track/1YqcGlCHNquxBhl...,151,NaN,NaN,200095,United States,1YqcGlCHNquxBhlUZsjhMT,[2uwXb02RFgtGOr6s6XMWlB],[],2oxjmU5c2mZzc7mSQqWTE9,2017-11-07,1,11,Tue
280597,2017-11-12,https://open.spotify.com/track/1YqcGlCHNquxBhl...,160,NaN,NaN,176687,United States,1YqcGlCHNquxBhlUZsjhMT,[2uwXb02RFgtGOr6s6XMWlB],[],2oxjmU5c2mZzc7mSQqWTE9,2017-11-07,1,11,Tue
280598,2017-11-13,https://open.spotify.com/track/1YqcGlCHNquxBhl...,170,NaN,NaN,190496,United States,1YqcGlCHNquxBhlUZsjhMT,[2uwXb02RFgtGOr6s6XMWlB],[],2oxjmU5c2mZzc7mSQqWTE9,2017-11-07,1,11,Tue


### Generate variables that change on a song-day basis

**Song_first_onchart**: a song's earliest appearance on the chart.

**Song_days_since_first**: the number of days between the current chart day and the song's first day on the chart.

**Song_days_onchart**: the actual number of days that the song has appeared on the chart.

**Song_days_since_release**: the number of days between the current chart day and the song's album's release date.

**Date_diff**: the number of days elapsed between a song's current appearance and its last appearance If Date_diff = 1, then the song appeared yesterday and today.

**Song_new_streak**: when Date_diff is > 1 , we assume that the song has temporarily dropped off the chart (hence ending its last consecutive streak), so we create a binary variable "Song_new_streak" that signals the beginnning of a new streak.

**Song_streak_id**: for each song's consecutive streak, assign a unique id by performing a cumulative sum of the 'Song_new_streak' variable.

**Song_consec_day**: for each consecutive day within a streak, assign a unique id.

In [81]:
track_days = data1[['Date', 'Track URI', 'Album Release']]\
.drop_duplicates().sort_values(by = ['Date', 'Track URI'])

track_days['Year_chart'] = track_days['Date'].dt.strftime('%Y').astype(int)
track_days['Month_chart'] = track_days['Date'].dt.strftime('%m')
track_days['Song_first_onchart'] = track_days.groupby(['Track URI'])[['Date']].transform('min')
track_days['Song_days_since_first'] = track_days['Date'] - track_days['Song_first_onchart']
track_days['Song_days_since_first'] = track_days['Song_days_since_first'].apply(lambda x: x.days)
track_days['Song_days_onchart'] = track_days.groupby(['Track URI']).cumcount() + 1
track_days['Song_days_since_release'] = track_days['Date'] - track_days['Album Release']
track_days['Song_days_since_release'] = track_days['Song_days_since_release'].apply(lambda x: x.days)

track_days['Date_lag'] = track_days.groupby(['Track URI'])[['Date']].shift(1)
track_days['Date_diff'] = track_days['Date'] - track_days['Date_lag']
track_days['Date_diff'] = track_days['Date_diff'].apply(lambda x: x.days)
track_days['Song_new_streak'] = np.where((track_days['Song_days_onchart']==1)|
                                         (track_days['Date_diff']>1), 
                                         1, 0)
track_days['Song_streak_id'] = track_days.groupby(['Track URI'])[['Song_new_streak']].transform('cumsum')
track_days['Song_consec_day'] = track_days.groupby(['Track URI', 'Song_streak_id']).cumcount() + 1

track_days

,Date,Track URI,Album Release,Year_chart,Month_chart,Song_first_onchart,Song_days_since_first,Song_days_onchart,Song_days_since_release,Date_lag,Date_diff,Song_new_streak,Song_streak_id,Song_consec_day
95101,2017-01-01,04CttTezSnv71USiiG9mIo,2016-11-11,2017,01,2017-01-01,0,1,51,NaT,NaN,1,1,1
175445,2017-01-01,04DwTuZ2VBdJCCC5TROn7L,2017-04-21,2017,01,2017-01-01,0,1,-110,NaT,NaN,1,1,1
47956,2017-01-01,05Z7jet4VDNVgNQWcYHnrk,2016-12-16,2017,01,2017-01-01,0,1,16,NaT,NaN,1,1,1
224461,2017-01-01,08WPvDEsHvTFuB9w8tC2OS,2017-07-21,2017,01,2017-01-01,0,1,-201,NaT,NaN,1,1,1
238912,2017-01-01,0B8B8cVRFIG1yznoQe7c9s,2016-11-04,2017,01,2017-01-01,0,1,58,NaT,NaN,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35991,2020-12-07,7vQbuQcyTflfCIOu3Uzzya,1957-12-02,2020,12,2018-11-15,753,54,23016,2020-12-06,1.0,0,2,12
120172,2020-12-07,7vrJn5hDSXRmdXoR30KgF1,2020-10-23,2020,12,2020-11-20,17,18,45,2020-12-06,1.0,0,1,18
119477,2020-12-07,7xapw9Oy21WpfEcib2ErSA,2011-01-01,2020,12,2018-11-15,753,107,3628,2020-12-06,1.0,0,5,24
178231,2020-12-07,7yiSvALPjMrBLDDrbcDRNy,2020-11-20,2020,12,2020-11-20,17,17,17,2020-12-06,1.0,0,2,3


## Convert list of collaborator URIs and names into long format

For each song/track URI, we expand/explode the list of Collaborator URI and Collaborator Name so that each artist can be on a separate row. This allows us to merge the characteristics of each artist later on. 

In [94]:
artist_uri = data1.drop(columns = ['Artist Name', 'Track URL', 'Artist', 'Album Release'])
artist_uri2 = artist_uri.explode('Artist URI', ignore_index=True)
artist_uri2[artist_uri2['No. of Artists']!=1]

,Date,Position,Track Name,Streams,Country,Track URI,Artist URI,Album URI,No. of Artists,Album_release_month,Album_release_dayweek
182,2020-07-10,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,5Mm2CJzNRiICC5MWRWQnBo,6PfSUFtkMVoDkx4MQkzOi3,0qnExDZfz0kVeBjixPsyjS,4,07,Fri
183,2020-07-10,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,5Mm2CJzNRiICC5MWRWQnBo,4UXqAaa6dQYAk18Lv7PEgX,0qnExDZfz0kVeBjixPsyjS,4,07,Fri
184,2020-07-10,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,5Mm2CJzNRiICC5MWRWQnBo,5aYf0AInMznHfXGaemKEBv,0qnExDZfz0kVeBjixPsyjS,4,07,Fri
185,2020-07-10,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,5Mm2CJzNRiICC5MWRWQnBo,0MfC3pip8rY8OFLJVVNvBO,0qnExDZfz0kVeBjixPsyjS,4,07,Fri
186,2017-01-01,187,Good Drank,146863,United States,39pS70eeDvyCAF3t8NAlVV,17lzZA2AlOHwCwFALHttmp,5vvvo79z68vWj9yimoygfS,3,06,Fri
...,...,...,...,...,...,...,...,...,...,...,...
437644,2020-10-01,192,Prospect (ft. Lil Baby),210504,United States,4iHSE5R1U8jf84tRn52xRt,5f7VJjfbwm532GiveGC0ZK,4Wb5bU9FkmZ84WkkL37rKA,2,06,Fri
437648,2019-09-27,166,Chicken Noodle Soup (feat. Becky G),257406,United States,6wyr4ReB05D9sJB1Rsmcqo,0b1sIQumIAsNbqAoIClSpy,76IRLp7YzBVLKsat6Ro9ae,2,09,Fri
437649,2019-09-27,166,Chicken Noodle Soup (feat. Becky G),257406,United States,6wyr4ReB05D9sJB1Rsmcqo,4obzFoKoKRHIphyHzJ35G3,76IRLp7YzBVLKsat6Ro9ae,2,09,Fri
437690,2017-03-17,196,Dennis Rodman,166943,United States,73cAKC1NbxHuFPcQ4slGtl,2o8lOQRjzsSC8UdbNN88HN,5WS1g0cKtjfK6eDoSLdv7d,2,03,Fri


In [96]:
artist_name = data1[['Date', 'Track URI', 'No. of Artists', 'Artist Name']]
artist_name2 = artist_name.explode('Artist Name', ignore_index=True)
artist_name2[artist_name2['No. of Artists']!=1]

,Date,Track URI,No. of Artists,Artist Name
182,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,100 gecs
183,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,Fall Out Boy
184,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,Craig Owens
185,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,Nicole Dollanganger
186,2017-01-01,39pS70eeDvyCAF3t8NAlVV,3,2 Chainz
...,...,...,...,...
437644,2020-10-01,4iHSE5R1U8jf84tRn52xRt,2,Lil Baby
437648,2019-09-27,6wyr4ReB05D9sJB1Rsmcqo,2,j-hope
437649,2019-09-27,6wyr4ReB05D9sJB1Rsmcqo,2,Becky G
437690,2017-03-17,73cAKC1NbxHuFPcQ4slGtl,2,mansionz


In [98]:
data2 = artist_name2.merge(artist_uri2, how = 'left', 
                           on = ['Date', 'Track URI', 'No. of Artists'],
                           left_index=True, right_index=True)\
.merge(artist_data, how = 'left', on = ['Artist URI'])
data2['Artist No.'] = data2.groupby(['Track URI', 'Date']).cumcount() + 1

In [99]:
data2[data2['No. of Artists']!=1]

,Date,Track URI,No. of Artists,Artist Name,Position,Track Name,Streams,Country,Artist URI,Album URI,Album_release_month,Album_release_dayweek,Artist Genre,Artist No.
182,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,100 gecs,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,6PfSUFtkMVoDkx4MQkzOi3,0qnExDZfz0kVeBjixPsyjS,07,Fri,"[glitchcore, hyperpop]",1
183,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,Fall Out Boy,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,4UXqAaa6dQYAk18Lv7PEgX,0qnExDZfz0kVeBjixPsyjS,07,Fri,"[emo, modern rock, pop punk]",2
184,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,Craig Owens,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,5aYf0AInMznHfXGaemKEBv,0qnExDZfz0kVeBjixPsyjS,07,Fri,[],3
185,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,Nicole Dollanganger,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,0MfC3pip8rY8OFLJVVNvBO,0qnExDZfz0kVeBjixPsyjS,07,Fri,[canadian singer-songwriter],4
186,2017-01-01,39pS70eeDvyCAF3t8NAlVV,3,2 Chainz,187,Good Drank,146863,United States,17lzZA2AlOHwCwFALHttmp,5vvvo79z68vWj9yimoygfS,06,Fri,"[atl hip hop, gangster rap, hip hop, pop rap, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437644,2020-10-01,4iHSE5R1U8jf84tRn52xRt,2,Lil Baby,192,Prospect (ft. Lil Baby),210504,United States,5f7VJjfbwm532GiveGC0ZK,4Wb5bU9FkmZ84WkkL37rKA,06,Fri,"[atl hip hop, atl trap, rap, trap]",2
437648,2019-09-27,6wyr4ReB05D9sJB1Rsmcqo,2,j-hope,166,Chicken Noodle Soup (feat. Becky G),257406,United States,0b1sIQumIAsNbqAoIClSpy,76IRLp7YzBVLKsat6Ro9ae,09,Fri,"[k-pop, k-rap]",1
437649,2019-09-27,6wyr4ReB05D9sJB1Rsmcqo,2,Becky G,166,Chicken Noodle Soup (feat. Becky G),257406,United States,4obzFoKoKRHIphyHzJ35G3,76IRLp7YzBVLKsat6Ro9ae,09,Fri,"[dance pop, latin, latin pop, latin viral pop,...",2
437690,2017-03-17,73cAKC1NbxHuFPcQ4slGtl,2,mansionz,196,Dennis Rodman,166943,United States,2o8lOQRjzsSC8UdbNN88HN,5WS1g0cKtjfK6eDoSLdv7d,03,Fri,[pop rap],1


### Generate variables that change on a artist-day basis

**Artist_first_onchart**: identify the first day an artist appears on the Top 200 chart.

**Artist_new_song**: generate a dummy variable that = 1 if this is the first time a track URI appears (Song_days_onchart == 1).

**Artist_new_collab**: generate a dummy variable that = 1 if this is the first time a track URI appears that is also a collab (No. of Collaborators != 1).

**Artist_new_solo**: generate a dummy variable that = 1 if this is the first time a track URI appears that is a solo (No. of Collaborators == 1).

**Artist_cumsum_songs**: calculate the cumulative sum of artist_new_song.

However, in the event that an artist has multiple songs on the chart on the same day, then this cumsum will be different for each song. To remedy this, we take the max cumsum per day for an artist to show how many songs an artist has on that day in total.

**Artist_cumu_songs**: the cumulative number of songs for an artist in a given day
(for each collaborator URI and date, find the max artist_cumsum_songs).

Analogously for Artist_cumu_collabs and Artist_cumu_solo.

Artist_cumu_songs should equal Artist_cumu_collabs + Artist_cumu_solo.

In [101]:
data2d = data2.merge(track_days, 
                      how= 'left', on = ['Date', 'Track URI'])\
.sort_values(by = ['Date', 'Track URI'])

data2d['Artist_first_onchart'] = data2d.groupby(['Artist URI'])[['Date']].transform('min')
data2d['Artist_new_song'] = np.where(data2d['Song_days_onchart']==1, 
                                     1, 0)

data2d['Artist_new_collab'] = np.where((data2d['Artist_new_song']==1) &
                                      (data2d['No. of Artists']!=1),
                                       1, 0)
data2d['Artist_new_solo'] = np.where((data2d['Artist_new_song']==1) &
                                      (data2d['No. of Artists']==1),
                                       1, 0)

data2d['Artist_cumsum_songs'] = data2d.groupby(['Artist URI'])[['Artist_new_song']]\
.transform('cumsum')
data2d['Artist_cumu_songs'] = data2d.groupby(['Artist URI', 'Date'])[['Artist_cumsum_songs']]\
.transform('max')

data2d['Artist_cumsum_collab'] = data2d.groupby(['Artist URI'])[['Artist_new_collab']]\
.transform('cumsum')
data2d['Artist_cumu_collab'] = data2d.groupby(['Artist URI', 'Date'])[['Artist_cumsum_collab']]\
.transform('max')

data2d['Artist_cumsum_solo'] = data2d.groupby(['Artist URI'])[['Artist_new_solo']]\
.transform('cumsum')
data2d['Artist_cumu_solo'] = data2d.groupby(['Artist URI', 'Date'])[['Artist_cumsum_solo']]\
.transform('max')

In [ ]:
# data2d

For each artist-day, find how many days the artist has been on the chart (cumulatively, not necessarily consecutively).

In [102]:
artist_days = data2d.loc[:, ['Artist URI', 'Date']]\
.sort_values(by=['Artist URI', 'Date'], ascending=[True,True]).drop_duplicates()
artist_days['Artist_days_onchart'] = artist_days.groupby(['Artist URI']).cumcount() + 1

artist_days

,Artist URI,Date,Artist_days_onchart
399078,002HSjuWsGMinkXTa7JcRp,2020-04-03,1
215451,00FQb4jTyendYWaN8pK0wa,2017-02-19,1
215452,00FQb4jTyendYWaN8pK0wa,2017-02-20,2
215453,00FQb4jTyendYWaN8pK0wa,2017-02-21,3
215454,00FQb4jTyendYWaN8pK0wa,2017-02-22,4
...,...,...,...
93782,7z5WFjZAIYejWy0NI5lv4T,2020-11-22,959
93783,7z5WFjZAIYejWy0NI5lv4T,2020-11-23,960
93786,7z5WFjZAIYejWy0NI5lv4T,2020-11-24,961
93789,7z5WFjZAIYejWy0NI5lv4T,2020-11-25,962


Merge data2d with artist days information.

In [103]:
data2e = data2d.merge(artist_days, how='left', on = ['Artist URI', 'Date'])

In [ ]:
# data2e

Drop intermediary (dummy) variables not useful for EDAs/model building.

In [104]:
data3 = data2e.drop(columns = ['Artist_new_song', 'Artist_cumsum_songs',
                               'Artist_new_collab', 'Artist_cumsum_collab',
                               'Artist_new_solo', 'Artist_cumsum_solo',
                               'Date_diff', 'Date_lag',
                               'Song_new_streak'])
# data3

For each track URI-date, find the average characteristics for all artists in the collaboration:

**Collab_avg_days_onchart**: average number of days on the chart of all collaborators

**Collab_avg_cumu_songs**: average number of songs on the chart of all collaborators

**Collab_avg_cumu_collab**: average number of previous collaborations of all collaborators

**Collab_avg_cumu_solo**: average number of previous solo songs released of all collaborators

In [105]:
data3b = data3

data3b['Collab_avg_days_onchart'] = data3.groupby(['Track URI', 'Date'])[['Artist_days_onchart']].transform('mean')
data3b['Collab_avg_cumu_songs'] = data3.groupby(['Track URI', 'Date'])[['Artist_cumu_songs']].transform('mean')
data3b['Collab_avg_cumu_collab'] = data3.groupby(['Track URI', 'Date'])[['Artist_cumu_collab']].transform('mean')
data3b['Collab_avg_cumu_solo'] = data3.groupby(['Track URI', 'Date'])[['Artist_cumu_solo']].transform('mean')

Reorder columns

In [116]:
cols1 = ['Date','Track Name', 'Streams', 'Position',
       'Artist Name', 'Artist No.', 'No. of Artists', 'Artist Genre']
cols2 = data3b.columns.difference(cols1, sort=False).tolist()
data3c = data3b[cols1 + cols2]

In [117]:
data3c

,Date,Track Name,Streams,Position,Artist Name,Artist No.,No. of Artists,Artist Genre,Track URI,Country,Artist URI,Album URI,Album_release_month,Album_release_dayweek,Album Release,Year_chart,Month_chart,Song_first_onchart,Song_days_since_first,Song_days_onchart,Song_days_since_release,Song_streak_id,Song_consec_day,Artist_first_onchart,Artist_cumu_songs,Artist_cumu_collab,Artist_cumu_solo,Artist_days_onchart,Collab_avg_days_onchart,Collab_avg_cumu_songs,Collab_avg_cumu_collab,Collab_avg_cumu_solo
0,2017-01-01,Lighthouse - Andrelli Remix,149929,183,Hearts & Colors,1,2,"[post-teen pop, viral pop]",04CttTezSnv71USiiG9mIo,United States,3wjsrpfO6odEphTZWx45RQ,4ywy3ahNM8FMH99Ueuf9ZA,11,Fri,2016-11-11,2017,01,2017-01-01,0,1,51,1,1,2017-01-01,1,1,0,1,1.0,1.000000,1.000000,0.0
1,2017-01-01,Lighthouse - Andrelli Remix,149929,183,Andrelli,2,2,[swedish pop],04CttTezSnv71USiiG9mIo,United States,5M2y5A6d5QZjw9JeKClagC,4ywy3ahNM8FMH99Ueuf9ZA,11,Fri,2016-11-11,2017,01,2017-01-01,0,1,51,1,1,2017-01-01,1,1,0,1,1.0,1.000000,1.000000,0.0
2,2017-01-01,In the Name of Love,435945,27,Martin Garrix,1,2,"[big room, dance pop, edm, electro house, pop,...",04DwTuZ2VBdJCCC5TROn7L,United States,60d24wfXkVzDSfLS6hyCjZ,75kX486cBBkuaLkZGjBptl,04,Fri,2017-04-21,2017,01,2017-01-01,0,1,-110,1,1,2017-01-01,1,1,0,1,1.0,2.000000,1.500000,0.5
3,2017-01-01,In the Name of Love,435945,27,Bebe Rexha,2,2,"[dance pop, electropop, pop, pop dance, post-t...",04DwTuZ2VBdJCCC5TROn7L,United States,64M6ah0SkkRsnPGtGiRAbb,75kX486cBBkuaLkZGjBptl,04,Fri,2017-04-21,2017,01,2017-01-01,0,1,-110,1,1,2017-01-01,3,2,1,1,1.0,2.000000,1.500000,0.5
4,2017-01-01,Party,151474,178,Chris Brown,1,3,"[dance pop, pop, pop rap, r&b, rap]",05Z7jet4VDNVgNQWcYHnrk,United States,7bXgB6jMjp9ATFy66eO08Z,35ljAE1f5Qmp2ZvVir34tL,12,Fri,2016-12-16,2017,01,2017-01-01,0,1,16,1,1,2017-01-01,1,1,0,1,1.0,1.666667,1.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437768,2020-12-07,BICHOTA,246194,178,KAROL G,1,1,"[latin, reggaeton, reggaeton colombiano, trap ...",7vrJn5hDSXRmdXoR30KgF1,United States,790FomKkXshlbRYZFtlgla,6LO6I2uJMkc0u7GHBYHa4Y,10,Fri,2020-10-23,2020,12,2020-11-20,17,18,45,1,18,2019-01-16,8,6,2,455,455.0,8.000000,6.000000,2.0
437769,2020-12-07,Mistletoe,590929,28,Justin Bieber,1,1,"[canadian pop, pop, post-teen pop]",7xapw9Oy21WpfEcib2ErSA,United States,1uNFoZAHBGtllmzznpCI3s,63MKT9hwmiMFFdFp5SdB1p,01,Sat,2011-01-01,2020,12,2018-11-15,753,107,3628,5,24,2017-01-01,56,39,17,1147,1147.0,56.000000,39.000000,17.0
437770,2020-12-07,Girls in the Hood,241171,184,Megan Thee Stallion,1,1,"[houston rap, pop, pop rap, trap queen]",7yiSvALPjMrBLDDrbcDRNy,United States,181bsRPaVXVlUKXrxwZfHK,0KjckH1EE6HRRurMIXSc0r,11,Fri,2020-11-20,2020,12,2020-11-20,17,17,17,2,3,2019-05-17,34,20,14,472,472.0,34.000000,20.000000,14.0
437771,2020-12-07,ROCKSTAR (feat. Roddy Ricch),521003,33,DaBaby,1,2,"[north carolina hip hop, rap]",7ytR5pFWmSjzHJIeQkgog4,United States,4r63FhuTkUYltbVAg5TQnk,623PL2MBg50Br5dLXC9E9e,04,Fri,2020-04-17,2020,12,2020-04-17,234,235,234,1,235,2019-03-30,75,51,24,588,634.5,54.500000,38.500000,16.0


Set index using Date, Track URI, and the ID of the collaborator in the track (Collab No.).
In the event that we need to pivot the dataframe to wide format (so that each artist's information is on a separate column), we can unstack using the Collab No.

In [118]:
data3d = data3c\
.set_index(['Date', 'Track URI', 'Artist No.'], append=False)
# data3c.loc[data3c['No. of Collaborators']!=1,:]

In [ ]:
# data3e = data3d.unstack(level=2)

In [ ]:
# data3f = data3e
# data3f.columns = [' '.join(col).strip() for col in data3f.columns.values]
# data3f

In [108]:
data3c.describe()

,No. of Artists,Position,Streams,Year_chart,Song_days_since_first,Song_days_onchart,Song_days_since_release,Song_streak_id,Song_consec_day,Artist_cumu_songs,Artist_cumu_collab,Artist_cumu_solo,Artist_days_onchart,Collab_avg_days_onchart,Collab_avg_cumu_songs,Collab_avg_cumu_collab,Collab_avg_cumu_solo
count,437773.000000,437773.000000,4.377730e+05,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000,437773.000000
mean,1.981906,99.664771,3.942047e+05,2018.436183,142.195887,123.136722,420.562709,2.614857,61.507336,29.969548,17.082049,12.887499,398.712008,398.712008,29.969548,17.082049,12.887499
std,1.092042,57.740146,2.746742e+05,1.114065,187.427879,158.630879,1996.005026,4.642557,77.042975,33.881362,20.394885,17.747958,326.403754,298.895935,29.783709,17.872877,15.394157
min,1.000000,1.000000,1.224880e+05,2017.000000,0.000000,1.000000,-208.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1.000000,49.000000,2.354340e+05,2017.000000,23.000000,21.000000,32.000000,1.000000,9.000000,5.000000,2.000000,1.000000,126.000000,153.400000,7.000000,3.250000,2.000000
50%,2.000000,99.000000,2.987920e+05,2018.000000,74.000000,66.000000,101.000000,1.000000,34.000000,18.000000,9.000000,6.000000,319.000000,342.000000,20.500000,11.000000,7.333333
75%,2.000000,150.000000,4.492710e+05,2019.000000,185.000000,163.000000,252.000000,3.000000,86.000000,43.000000,25.000000,18.000000,605.000000,581.000000,42.000000,25.000000,18.000000
max,22.000000,200.000000,5.749019e+06,2020.000000,1436.000000,1390.000000,28830.000000,100.000000,676.000000,216.000000,127.000000,103.000000,1403.000000,1403.000000,213.000000,124.000000,103.000000


In [109]:
data3c.loc[data3c['No. of Artists']!=1, :].describe()

,No. of Artists,Position,Streams,Year_chart,Song_days_since_first,Song_days_onchart,Song_days_since_release,Song_streak_id,Song_consec_day,Artist_cumu_songs,Artist_cumu_collab,Artist_cumu_solo,Artist_days_onchart,Collab_avg_days_onchart,Collab_avg_cumu_songs,Collab_avg_cumu_collab,Collab_avg_cumu_solo
count,273601.000000,273601.000000,2.736010e+05,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000,273601.000000
mean,2.571091,97.780392,3.967226e+05,2018.396928,125.192181,111.680973,272.488781,2.254564,61.033154,29.508039,19.014814,10.493226,396.859248,396.859248,29.508039,19.014814,10.493226
std,0.991199,57.674022,2.654753e+05,1.108384,160.955100,138.674204,1504.023301,3.878399,73.752406,34.556997,21.231499,17.204197,327.786634,282.705348,27.870607,17.214930,13.083007
min,2.000000,1.000000,1.254840e+05,2017.000000,0.000000,1.000000,-194.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,2.000000,48.000000,2.367290e+05,2017.000000,22.000000,21.000000,30.000000,1.000000,10.000000,4.000000,3.000000,0.000000,124.000000,169.000000,9.000000,5.750000,1.333333
50%,2.000000,96.000000,3.039440e+05,2018.000000,69.000000,63.000000,88.000000,1.000000,35.000000,17.000000,11.000000,3.000000,314.000000,348.000000,20.666667,14.000000,6.000000
75%,3.000000,148.000000,4.590750e+05,2019.000000,164.000000,150.000000,208.000000,2.000000,86.000000,42.000000,27.000000,14.000000,606.000000,570.500000,41.500000,27.250000,14.000000
max,22.000000,200.000000,4.444027e+06,2020.000000,1315.000000,1137.000000,28830.000000,89.000000,676.000000,216.000000,127.000000,103.000000,1403.000000,1384.500000,204.000000,108.000000,96.000000


Export dataframe with collaborations only

In [119]:
data3c.loc[data3c['No. of Artists']!=1,:]\
.to_csv('../output/2021.01.04 spotify_us_collab_fe.csv')

Export dataframe with all songs in the US

In [120]:
data3c\
.to_csv('../output/2021.01.04 spotify_us_all_fe.csv')